In [31]:
import pandas as pd
import numpy as np
import os

In [41]:
# Get project paths
project_root = os.path.dirname(os.getcwd())

fall_18_data_path = project_root + '/Fall_2018/'
fall_19_data_path = project_root + '/Fall_2019/'
pre_processing_path = project_root + '/Pre-Processing/'

In [42]:
# Read messages data from fall 2018 and fall 2019
df_fall_18_messages = pd.read_excel(pre_processing_path + 'FA18_all_messages.xlsx', index_col=0)
df_fall_19_messages = pd.read_excel(pre_processing_path + 'FA19_all_messages.xlsx', index_col=0)
print("FALL 2018:\n", df_fall_18_messages.head())
print("FALL 2019:\n", df_fall_19_messages.head())

FALL 2018:
    User#    UserID    Group#  \
0      6  42683026  44081449   
1      0  25501571  44081449   
2      6  42683026  44081449   
3      0  25501571  44081449   
4      0  25501571  44081449   

                                                Text            Timestamp  
0  Hey @Katie Poteet I know you said we should em...  2018-09-17 19:20:59  
1  @Mary Cassell I would email Dr. K anyway with ...  2018-09-17 19:22:12  
2  Ok, thanks a lot. I have the email typed but w...  2018-09-17 19:24:07  
3                     Upstairs from our lecture hall  2018-09-17 20:59:57  
4                                     Sherman 207!!!  2018-09-17 20:59:59  
FALL 2019:
    User#    UserID    Group#  \
0      2  74779648  52791846   
1      5  29836867  52791846   
2      1  44467665  52791846   
3      1  44467665  52791846   
4      1  44467665  52791846   

                                                Text            Timestamp  
0  Hey where can I find the essays to comment on ...  2019

In [43]:
# Read user-id-to-name files 
df_fall_18_users_and_ids = pd.read_excel(pre_processing_path + 'FA18_users_and_ids.xlsx', index_col=0)
df_fall_19_users_and_ids = pd.read_excel(pre_processing_path + 'FA19_users_and_ids.xlsx', index_col=0)
print("FALL 2018:\n",df_fall_18_users_and_ids.head())
print("FALL 2019:\n",df_fall_19_users_and_ids.head())

FALL 2018:
                       User ID
Last Name + Initial          
Abiog T.             36071565
Akanoh S.            52309964
Allman M.            63282225
Anderson C.          21708768
Andre C.             32072158
FALL 2019:
                    Name
User ID                
73598857   Frampton, R.
35809612  Slaughter, C.
74600149    Karanja, E.
72761543      Qamar, B.
41840220         Fu, A.


In [88]:
# Read student information files to obtain race and gender information
df_fall_18_student_info = pd.read_excel(pre_processing_path + 'FA18_student_information.xlsx', index_col=0)
df_fall_19_student_info = pd.read_excel(pre_processing_path + 'FA19_student_information.xlsx', index_col=0)
print("FALL 2018:\n",df_fall_18_student_info.head())
print("FALL 2019:\n",df_fall_19_student_info.head())

FALL 2018:
           Last Name First Name Last Name + Initial Gender   Race    Major
Team                                                                     
Isochrone     Abiog  Theo Karl            Abiog T.      M  ASIAN  PCHE BS
Mercury      Akanoh  Stephanie           Akanoh S.      F  BLACK  BIOL BS
Zodiac       Allman    Michael           Allman M.      M  WHITE   GES BS
Sirius     Anderson    Cameron         Anderson C.      F  BLACK  CHEM BS
Milky Way     Andre      Chase            Andre C.      M  WHITE  BIOL BS
FALL 2019:
           Last Name First Name Gender  Race            Major
Team                                                        
Nutmeg  Christenson        Sam      F   NaN             BIOC
Nutmeg     Courduff      Allie      F   NaN          BIOL BS
Nutmeg         Imam     Danyah      F   NaN  BIOL BS/PSYC BS
Nutmeg          Lee     Joseph      M   NaN     MATH BS/PCMS
Nutmeg  Majekodunmi       Tobi      M   NaN        PHYS/PMEE


In [89]:
merged_id_race_gender = pd.merge(left=df_fall_18_users_and_ids, right=df_fall_18_student_info, how='left', left_on='Last Name + Initial', right_on='Last Name + Initial')

In [90]:
merged_id_race_gender.to_excel(pre_processing_path + "FA18_merged_id_race_gender.xlsx")  

In [91]:
# Read corrected file (added race and gender of known team leaders, hence corrected some unknowns)
df_fall_18_id_race_gender = pd.read_excel(pre_processing_path + "FA18_corrected_merged_id_race_gender.xlsx")

In [93]:
df_messages_labeled_FA18 = pd.DataFrame()

# Create dictionary to match user_id to race and gender
race_dictionary = dict(zip(df_fall_18_id_race_gender['User ID'].astype('int').values, df_fall_18_id_race_gender['Race']))
gender_dictionary = dict(zip(df_fall_18_id_race_gender['User ID'].astype('int'), df_fall_18_id_race_gender['Gender']))

race = []
gender = []

# Create columns for race and gender
for i in range(len(df_fall_18_messages["Text"])):
    if (df_fall_18_messages["UserID"][i] != "calendar"):
        user_id = int(df_fall_18_messages["UserID"][i])
        if user_id in race_dictionary.keys():
            race.append(race_dictionary[user_id])
        else:
            race.append("Unknown")
        if user_id in gender_dictionary.keys():
            gender.append(gender_dictionary[user_id])
        else:
            gender.append("Unknown")
    else:
        race.append("Unknown")
        gender.append("Unknown")

# append user id, sender race, sender gender, and text columns to dataframe, export to excel
df_messages_labeled_FA18["User ID"] = df_fall_18_messages['UserID']
df_messages_labeled_FA18["Race"] = pd.DataFrame(race)
df_messages_labeled_FA18["Gender"] = pd.DataFrame(gender)
df_messages_labeled_FA18["Text"] = df_fall_18_messages["Text"]
df_messages_labeled_FA18.to_excel(fall_18_data_path + "messages_with_race_gender_FA18.xlsx")